# First lets process the stylish data

In [1]:
import os
import pandas as pd

In [2]:
os.listdir('/l/disk1/awstebas/data/TTS/speaker-blizzard-2021-es/interrogativas_sn/transcricao')

['interrogativas_sn.ref', 'interrogativas_sn.txt']

In [3]:
os.listdir('/l/disk1/awstebas/data/LJSpeech-1.1')

['metadata.csv', 'wavs', 'README']

In [4]:
eps_paths = []

for f in os.listdir('/l/disk1/awstebas/data/TTS/speaker-blizzard-2021-es/'):
#     if(f in ['speaker-melissa', 'speaker-paola'] ):
#         for folder in os.listdir(f'/l/disk1/awstebas/data/TTS/{f}'):
#     if("eps_neutro" in f or "eps_animado" in f or "eps_rispido" in f or 'eps_acolhedor' in f):
#         if(f[:4] != "sint"):
        if(('.' not in f) & ('profiles' not in f)&('tool_gerabase' not in f)):
            eps_paths.append(f)

In [5]:
# The code above should result in this cell output. Maybe you should change the above code
eps_paths

['interrogativas_qu', 'interrogativas_sn', 'declarativas']

In [6]:
data_p = '/l/disk1/awstebas/data/TTS/speaker-blizzard-2021-es/'

In [7]:
## CHECKING FOR WAV22 FOLDER IN EACH STYLE
c = True
t = True
for e in eps_paths:
    if('wav22' not in os.listdir(data_p + e)):
        print("There is no 'wav22' folder in ", e)
        c = False
    if('transcricao' not in os.listdir(data_p + e)):
        print("There is no 'transcricao' folder in ", e)
        t = False
        
if(c == True):
    print("All folders have wav22 file")

if(t == True):
    print("All folders have transcricao file")

All folders have wav22 file
All folders have transcricao file


In [8]:
## CHECKING TEXT STRUCTURE

ex = '/l/disk1/awstebas/data/TTS/speaker-adriana/eps_rispido_aco/transcricao/eps_rispido_aco.txt'

with open(ex, 'r') as f:
    for line in f.readlines():
        print(line.split(':'))
        break

['eps_rispido_aco0001', ' A pergunta que resta é se houve ou não suborno.\n']


In [9]:
## defining get_wav and text

styles = []
wavs = []
texts = []
for e in eps_paths:
    local_dir = data_p + e
    
    transc_path = local_dir + '/transcricao/' + e + '.txt'
    
#     if(os.path.isfile(transc_path)):
    with open(transc_path, 'r') as f:
        for line in f.readlines():
            splitted = line.split(':')

            wpath = local_dir + '/wav22/' + splitted[0] + '.wav'

            if(os.path.isfile(wpath)):
                wavs.append(wpath)

                texts.append(splitted[1][1:-1])

                style = 'spanish'
                styles.append(style)
            else:
                print(f'{wpath} is not a file')

In [10]:
len(texts)

5181

In [1]:
import os
def get_phones(txt):
    
    with open('tmp.txt', "w", encoding="utf-8") as f:
        f.writelines(txt)
    
    cmd = '/workspace/tool_language/tool_language -l /workspace/tool_language/libptbr.so.4.6.0 -p /workspace/tool_language/ -i tmp.txt -o tmp_pnh.txt --phonemes -s'
    
    os.system(cmd)
    
    with open ('tmp_pnh.txt', "r") as f:
        lines = f.readlines()
    
    return lines[0][:-1]

In [6]:
get_phones('''The intermediate features S are passed through the Decoder consisting of two linear layers, one of output size 192 and another for a linear transformation from 192 to the final number of classes N, to compute the probability that the current segment belongs to a speaker from the training set.''')

"#p 'ee ss ts ic 'eh un ee zz 'en pp ll uc dz ic ss 'in tt ee zz ic zh ee rr 'aa dd ac pp 'ee ll uc mm 'ee uw 'uu uw ts ic mm uc mm oo dd 'ee ll uc #p"

/bin/sh: 1: /workspace/tool_language/tool_language: not found


In [13]:
phones = []

for t in texts:
    phones.append(get_phones(t))

In [14]:
speakers = ['blizzard_es'] * len(phones)
df = pd.DataFrame({'norm_text': texts,
                   'phonetic_transcription': phones,
                   'wav_path': wavs,
                   'style': styles,
                   'speaker': speakers})

df.head(), df.shape

(                                           norm_text  \
 0                                  ¿en quién podría?   
 1           ¿Cuál es la peor película que has visto?   
 2  ¿Qué es algo que siempre se busca, nunca se en...   
 3                ¿Cómo entró por la puerta de abajo?   
 4  ¿Qué le ofrece la poesía en comparación con la...   
 
                               phonetic_transcription  \
 0             #p 'en kk ii 'en pp oo dd rd 'ii ac #q   
 1  #p kk uw 'aa uw ic zz ll ac pp ee 'oo rf pp ee...   
 2  #p kk 'eh ic zz 'aa uw gg uc kk ic ss ii 'en p...   
 3  #p kk 'oh mm uc en tt rd 'oh pp oo rf ll ac pp...   
 4  #p kk 'eh ll 'ee oo ff rd 'eh ss ic ll ac pp o...   
 
                                             wav_path    style      speaker  
 0  /l/disk1/awstebas/data/TTS/speaker-blizzard-20...  spanish  blizzard_es  
 1  /l/disk1/awstebas/data/TTS/speaker-blizzard-20...  spanish  blizzard_es  
 2  /l/disk1/awstebas/data/TTS/speaker-blizzard-20...  spanish  blizzard_es  


In [15]:
df['phonetic_transcription'].values[:3]

array(["#p 'en kk ii 'en pp oo dd rd 'ii ac #q",
       "#p kk uw 'aa uw ic zz ll ac pp ee 'oo rf pp ee ll 'ii kk uc ll ac kk ic ac zz vv 'ii ss tt uc #q",
       "#p kk 'eh ic zz 'aa uw gg uc kk ic ss ii 'en pp rd ic ss ic bb 'uu ss kk ac #c nn 'un kk ac ss ic en kk uw 'en tt rd ac 'ii pp ss ic ll on tt 'oo dd uc zz dd ee ss pp ee rf dz ii ss ii 'an mm uc ss #q"],
      dtype=object)

In [16]:
import IPython
k = 3
print(df['phonetic_transcription'].values[k], df['norm_text'].values[k])
IPython.display.Audio(df['wav_path'].values[k])

#p kk 'oh mm uc en tt rd 'oh pp oo rf ll ac pp uu 'eh rf tt ac dz ic aa bb 'aa zh uc #q ¿Cómo entró por la puerta de abajo?


In [17]:
# Now lets take some samples from LJSpeech
LJ_BASE_PATH = '/l/disk1/awstebas/data/LJSpeech-1.1'
os.listdir(LJ_BASE_PATH)

['metadata.csv', 'wavs', 'README']

In [18]:
lj_df = pd.read_csv(LJ_BASE_PATH + '/metadata.csv', sep = '|', names=['wav_path', 'text', 'norm_text'])
lj_df.head()

,wav_path,text,norm_text
0,LJ001-0001,"Printing, in the only sense with which we are ...","Printing, in the only sense with which we are ..."
1,LJ001-0002,in being comparatively modern.,in being comparatively modern.
2,LJ001-0003,For although the Chinese took impressions from...,For although the Chinese took impressions from...
3,LJ001-0004,"produced the block books, which were the immed...","produced the block books, which were the immed..."
4,LJ001-0005,the invention of movable metal letters in the ...,the invention of movable metal letters in the ...


In [19]:
lj_df =lj_df.sample(5000, random_state = 42).reset_index(drop=True)

In [20]:
lj_df.shape

(5000, 3)

In [21]:
get_phones(lj_df.norm_text[13])

"#p dd ee rx ee kk on ss tt rd uu kk ts ii 'on 'uu ac zz dd ee mm ii nn ii mm 'un ts 'ii mm ic #p aa kk oo rf dz 'in gg ll ic #c sh ic pp rd oo bb 'aa bb ll ic mm 'eh ts ic 'oo zz vv ac uw dz ic 'eh ts ic aa bb 'aa uw ts ic tt uu 'eh uw vv ic #c dd 'oo ij ss pp 'on tt uc ss #c ts 'ii rf ts ic tt 'uu uw #c"

In [22]:
lj_df['norm_text'][123]

nan

In [23]:
texts = []
phones = []
wavs = []

for i in range(lj_df.shape[0]):
    try:
        phon = get_phones(lj_df.norm_text.values[i])
        texts.append(lj_df.norm_text.values[i])
#         print(texts[-1], i)
        phones.append(phon)
        wavs.append(LJ_BASE_PATH + '/wavs/' + lj_df.wav_path.values[i] + '.wav')
    except:
        continue

In [24]:
speakers = ['ljspeech'] * len(phones)
styles = ['english'] * len(phones)
df_ljspeech = pd.DataFrame({'norm_text': texts,
                   'phonetic_transcription': phones,
                   'wav_path': wavs,
                   'style': styles,
                   'speaker': speakers})

df_ljspeech.head(), df_ljspeech.shape

(                                           norm_text  \
 0  At that station the safes were given out, heav...   
 1               No traces of its moat have appeared.   
 2  A notorious miser, Robert Smith, had recently ...   
 3        No attempt was made to maintain discipline.   
 4  Few realizing that the dreadful fate would ove...   
 
                               phonetic_transcription  \
 0  #p 'eh ts ic tt 'aa ts ic ii ss tt 'ee ij sh a...   
 1  #p nn uc tt rd 'aa ss ic zz 'oh ff ic 'ii tt z...   
 2  #p ac nn oo tt oo rr ii 'oo uw zz mm ii zz 'ee...   
 3  #p nn uc aa tt 'en pp ts ic 'uu ac zz mm 'ee i...   
 4  #p ff 'ee uw rx ee aa ll 'ii zz in tt 'aa ts i...   
 
                                             wav_path    style   speaker  
 0  /l/disk1/awstebas/data/LJSpeech-1.1/wavs/LJ015...  english  ljspeech  
 1  /l/disk1/awstebas/data/LJSpeech-1.1/wavs/LJ028...  english  ljspeech  
 2  /l/disk1/awstebas/data/LJSpeech-1.1/wavs/LJ013...  english  ljspeech  
 3  /l/disk1

In [25]:
df_ljspeech.head(1)

,norm_text,phonetic_transcription,wav_path,style,speaker
0,"At that station the safes were given out, heav...",#p 'eh ts ic tt 'aa ts ic ii ss tt 'ee ij sh a...,/l/disk1/awstebas/data/LJSpeech-1.1/wavs/LJ015...,english,ljspeech


In [26]:
os.path.isfile(df_ljspeech.wav_path[0])

True

In [27]:
df_full = pd.concat([df, df_ljspeech]).reset_index(drop=True)

In [28]:
# Checking wavs consistency

errors = 0
for wpath in df_full.wav_path:
    if(not os.path.isfile(wpath)):
        errors += 1
        print(wpath)

In [29]:
import IPython
k = -3
print(df_full['phonetic_transcription'].values[k], df_full['norm_text'].values[k])
IPython.display.Audio(df_full['wav_path'].values[k])

#p 'ii ts ic 'uu ac zz in tt 'ee zz ic zh aa 'ii uw ss #c uu ij tt dd rd aa 'uu nn ic ff rd 'on dd ee pp rd ee ss 'uu rr ic 'oh ff ic aa uw tt oo rr 'ii ts ic #c tt 'aa ts ic dd ee nn 'ii uw rx 'uu ll ic zz uu 'eh rr ic in vv aa rr ii 'aa bb ll ic ii gg nn oo rr 'eh dz ic #p It was in these jails, withdrawn from the pressure of authority, that the new rules were invariably ignored.


# Now lets define our "rest" dataset and split in train and val randomly

In [ ]:
# total_to_exclude = []
# total_to_exclude.extend(nonpaired)
# total_to_exclude.extend(paired)

# df_rest = df[~df.phonetic_transcription.isin(total_to_exclude)]
# df_rest.shape, df.shape

In [30]:
from sklearn.model_selection import train_test_split

In [32]:
df_train, df_val = train_test_split(df_full, test_size = 0.2, stratify=df_full['style'], random_state = 42)

In [33]:
df_train.shape, df_val.shape

((8140, 5), (2035, 5))

In [34]:
df_val, df_test = train_test_split(df_val, test_size = 0.2, stratify=df_val['style'], random_state = 42)

In [35]:
df_val.shape, df_test.shape

((1628, 5), (407, 5))

In [36]:
import librosa

def get_durations(wpath):
    y, sr = librosa.load(wpath, sr = None)
    return len(y)/sr

In [37]:
durs = []
for w in df_train['wav_path']:
    durs.append(get_durations(w))
df_train['durs'] = durs

durs = []
for w in df_val['wav_path']:
    durs.append(get_durations(w))
df_val['durs'] = durs

In [38]:
df_train.durs.sum()/3600, df_val.durs.sum()/3600

(11.962277865961198, 2.433248425296044)

In [39]:
# df_train per speaker
df_train.groupby('style').agg({'durs': 'sum'})/3600

,durs
style,
english,7.293088
spanish,4.669190


In [40]:
# df_val per speaker
df_val.groupby('style').agg({'durs': 'sum'})/3600

,durs
style,
english,1.486887
spanish,0.946362


# Now, getting the train and val dataset from universal list and getting only 3 speakers data

In [41]:
u_train = pd.read_csv("universal_cpqd_train.csv", encoding= 'utf-8', sep=';')
u_val = pd.read_csv("universal_cpqd_val.csv", encoding= 'utf-8', sep=';')

In [42]:
u_train['style'] = 'neutro'
u_val['style'] = 'neutro'

speakers_in = ['adriana','chiquinho','rosana']

u_train = u_train[u_train['speaker'].isin(speakers_in)]
u_val = u_val[u_val['speaker'].isin(speakers_in)]

u_train.speaker.unique(), u_val.speaker.unique()

(array(['adriana', 'chiquinho', 'rosana'], dtype=object),
 array(['adriana', 'chiquinho', 'rosana'], dtype=object))

In [43]:
def get_rec(wpath):
    return wpath.split('/')[7]

recs = []
for w in u_train.wav_path:
    recs.append(get_rec(w))
u_train['rec'] = recs

recs = []
for w in u_val.wav_path:
    recs.append(get_rec(w))
u_val['rec'] = recs

In [44]:
u_train.rec.unique(), u_val.rec.unique()

(array(['aec', 'akiva', 'akiva_fidelizacao', 'akiva_portabilidade',
        'aplicacao', 'atendimento', 'avm', 'avm_custom', 'bradesco',
        'bradesco_bia', 'bradesco_ura', 'callcenter', 'chat_covid',
        'comemorativas', 'datas', 'digitos', 'dnk_expr',
        'elev_dentalprime', 'emocao', 'empresas', 'enderecos',
        'freqpalavras', 'horas', 'ibm', 'ibm_fastshop', 'ibm_tim',
        'interrogativas', 'interrogativas_ou', 'interrogativas_qu',
        'interrogativas_sn', 'invenit', 'marketing_cpqd', 'neutro',
        'nomes', 'nomes_freq', 'nomes_int', 'numeros', 'pg_amc',
        'pg_americo', 'pg_avon', 'pg_demoKami_v2', 'pg_demoPrioritario',
        'pg_flex', 'pg_generico', 'pg_interrogativas', 'pg_localcred',
        'pg_meucartao', 'pg_renner', 'pg_riachuelo',
        'pg_riachuelo_expressivo', 'pg_vivo', 'quantias', 'renaut',
        'riqueza_fonetica', 'robbu', 'saudacoes', 'siglas',
        'sulamerica_auto', 'sulamerica_saude', 'sulamerica_vida',
        'textofa

In [45]:
u_train.shape , u_train[u_train.rec.isin(['rf_selecionadas'])].shape

((22807, 5), (1905, 5))

In [46]:
u_train[(u_train.rec.isin(['riqueza_fonetica']))].speaker.value_counts()

adriana      2157
chiquinho    1870
Name: speaker, dtype: int64

In [47]:
u_train[(u_train['speaker'].isin(['rosana','chiquinho'])) | ((u_train['speaker'] == 'adriana') & (u_train['rec'] == 'riqueza_fonetica'))].speaker.value_counts()

rosana       6684
chiquinho    4680
adriana      2157
Name: speaker, dtype: int64

In [48]:
u_train = u_train[(u_train['speaker'].isin(['rosana','chiquinho'])) | ((u_train['speaker'] == 'adriana') & (u_train['rec'] == 'riqueza_fonetica'))]

In [49]:
u_val.speaker.value_counts()

adriana      108
rosana        50
chiquinho     30
Name: speaker, dtype: int64

In [50]:
## Getting only the necessary cols
cols = ['phonetic_transcription', 'wav_path', 'speaker','style']

u_train = u_train[cols]
u_val = u_val[cols]

df_train = df_train[cols]
df_val = df_val[cols]

df_train_tot = pd.concat([df_train, u_train]).reset_index(drop = True)
df_val_tot = pd.concat([df_val, u_val]).reset_index(drop = True)

df_train_tot.shape, df_val_tot.shape

((21661, 4), (1816, 4))

In [51]:
df_train_tot.head()

,phonetic_transcription,wav_path,speaker,style
0,#p aa zz 'oh tt ee mm ic #c gg oo uw pp 'eh ee...,/l/disk1/awstebas/data/TTS/speaker-blizzard-20...,blizzard_es,spanish
1,#p nn uc kk ii 'ee rr uc kk ic 'oo tt rd uc ee...,/l/disk1/awstebas/data/TTS/speaker-blizzard-20...,blizzard_es,spanish
2,#p kk rd 'ee uw kk ic ll uc ss ss 'aa bb ac dd...,/l/disk1/awstebas/data/TTS/speaker-blizzard-20...,blizzard_es,spanish
3,#p ss 'ii pp rd ee gg 'un tt ac ac kk 'aa dd a...,/l/disk1/awstebas/data/TTS/speaker-blizzard-20...,blizzard_es,spanish
4,#p dd ee ll 'on gg ic 'en dz ic bb 'uu uw kk i...,/l/disk1/awstebas/data/LJSpeech-1.1/wavs/LJ033...,ljspeech,english


In [52]:
df_train_tot.speaker.value_counts()

rosana         6684
chiquinho      4680
blizzard_es    4145
ljspeech       3995
adriana        2157
Name: speaker, dtype: int64

In [53]:
df_val_tot.speaker.value_counts()

blizzard_es    829
ljspeech       799
adriana        108
rosana          50
chiquinho       30
Name: speaker, dtype: int64

In [54]:
durs = []
for w in df_train_tot['wav_path']:
    durs.append(get_durations(w))
df_train_tot['durs'] = durs

durs = []
for w in df_val_tot['wav_path']:
    durs.append(get_durations(w))
df_val_tot['durs'] = durs

In [55]:
df_train_tot.durs.sum()/3600, df_val_tot.durs.sum()/3600

(25.893488372385995, 2.651888170823885)

In [56]:
# df_train per speaker
df_train_tot.groupby('style').agg({'durs': 'sum'})/3600

,durs
style,
english,7.293088
neutro,13.931211
spanish,4.669190


In [1]:
# df_train per speaker
df_train_tot.groupby('speaker').agg({'durs': 'sum'})/3600

NameError: name 'df_train_tot' is not defined

In [58]:
cols = ['phonetic_transcription', 'wav_path', 'speaker','style']

df_train_tot[cols].to_csv('mp_languages_train.csv', index = False, sep=';', encoding = 'utf-8')
df_val_tot[cols].to_csv('mp_languages_val.csv', index = False, sep=';', encoding = 'utf-8')
df_test[cols].to_csv('mp_languages_test.csv', index = False, sep=';', encoding = 'utf-8')

In [4]:
import pandas as pd
df_train_tot = pd.read_csv('mp_languages_train.csv', delimiter= ';', encoding = 'utf-8')
df_val_tot = pd.read_csv('mp_languages_val.csv', delimiter= ';',  encoding = 'utf-8')
df_test_tot = pd.read_csv('mp_languages_test.csv', delimiter= ';',  encoding = 'utf-8')

In [6]:
df_train_tot.head()

,phonetic_transcription,wav_path,speaker,style
0,#p aa zz 'oh tt ee mm ic #c gg oo uw pp 'eh ee...,/l/disk1/awstebas/data/TTS/speaker-blizzard-20...,blizzard_es,spanish
1,#p nn uc kk ii 'ee rr uc kk ic 'oo tt rd uc ee...,/l/disk1/awstebas/data/TTS/speaker-blizzard-20...,blizzard_es,spanish
2,#p kk rd 'ee uw kk ic ll uc ss ss 'aa bb ac dd...,/l/disk1/awstebas/data/TTS/speaker-blizzard-20...,blizzard_es,spanish
3,#p ss 'ii pp rd ee gg 'un tt ac ac kk 'aa dd a...,/l/disk1/awstebas/data/TTS/speaker-blizzard-20...,blizzard_es,spanish
4,#p dd ee ll 'on gg ic 'en dz ic bb 'uu uw kk i...,/l/disk1/awstebas/data/LJSpeech-1.1/wavs/LJ033...,ljspeech,english


In [7]:
df_train_tot['style'].value_counts()

neutro     13521
spanish     4145
english     3995
Name: style, dtype: int64

In [15]:
tot_tokens = []

def augment(tot_tokens, new_tokens):
    tot_tokens.extend(new_tokens)
    
    return list(set(tot_tokens))

for p in df_train_tot.phonetic_transcription:
    tot_tokens.extend(p.split())

In [20]:
import numpy as np

' '.join(np.unique(tot_tokens))

"#c #d #e #k #p #q #s 'aa 'an 'ee 'eh 'en 'ii 'in 'oh 'on 'oo 'un 'uu aa ac an bb dd dz ee eh en ff gg ic ii ij in jn kk lh ll mm nh nn oh on oo pp rd rf rr rx sh ss ts tt uc un uu uw vv wn zh zz"

In [21]:
OLD = "| <blank> <unk> ss ac #p uc ic tt kk 'aa aa ee dd pt_BR zz mm pp uw rr nn ii 'ee 'ii ll oo rf rd dz vv 'eh ij 'an 'oo #c bb ff ts gg 'en en 'oh wn rx 'uu zh #q uu on an in 'in 'on sh nh un lh 'un jn #k eh oh #e #s #d <sos/eos>"

In [22]:
for p in OLD.split():
    if p not in np.unique(tot_tokens):
        print(p)

|
<blank>
<unk>
pt_BR
<sos/eos>
